# *Credit Card Fraud Detection - Task 1*

![logo](images/logo.jpg)

In [ ]:
import geodatasets
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sb
import warnings

from geopandas import GeoDataFrame
from shapely.geometry import Point
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import kurtosis, norm, normaltest, shapiro, skew

%matplotlib inline

warnings.filterwarnings('ignore')

## Compreensão do Tema

### Compreensão do Tema

Uma fraude de cartão de crédito ocorre quando um indivíduo não autorizado acede à informação do cartão de crédito de outra pessoa para fazer compras em seu nome, indevidamente.

Estas fraudes são um problema cada vez mais comum e recorrente na atualidade, pelo que todas as entidades envolvidas investem significativamente para descobrir padrões e indicadores que podem indiciar situações ou tentativas de fraude.

### Determinação dos Objetivos do Tema

De maneira a ir ao encontro das necessidades dos bancos, reguladores e fornecedores de cartões de crédito, é essencial conseguir prever corretamente se uma transação financeira é, com alta probabilidade, fraudulenta.

Como tal, torna-se extremamente útil desenvolver um modelo que identifique eficazmente transações suspeitas de fraude, de maneira a aumentar a segurança financeira e a reduzir as atividades fraudulentas.

### Avaliação da Situação

Os dados disponíveis para análise foram fornecidos por uma empresa de cartões de crédito, correspondendo a informação de 2023, com registos sobre clientes e comerciantes.

O conjunto de dados inclui informação detalhada sobre cidades, clientes, comerciantes e transações, estando cada transação detalhada como legítima ou fraudulenta, permitindo uma abordagem de aprendizagem supervisionada para identificar padrões indicadores de fraude.

### Determinação dos Objetivos de Mineração de Dados

O problema consiste em prever se uma transação é, ou não, fraudulenta, tendo como base um conjunto de dados associados à transação, ou seja, classificar cada transação como pertencente a um de dois grupos, mutuamente exclusivos: fraudulenta ou legítima (não fraudulenta).

Para resolver o problema, deve ser desenvolvido um modelo de aprendizagem computacional supervisionada com capacidade para efetuar uma classificação binária que distinga os dois casos possíveis para cada transação.

## Compreensão dos Dados

### Recolha dos Dados Iniciais

O conjunto de dados fornecido é constituído por quatro ficheiros, dividindo a informação da forma abaixo exposta.

1. **cities.csv:** informação sobre várias cidades
2. **customers.csv:** informação demográfica de cada cliente
3. **merchants.csv:** detalhes de cada comerciante
4. **transactions.csv:** registos de transações individuais

In [ ]:
cities = pd.read_csv('data/cities.csv')
customers = pd.read_csv('data/customers.csv')
merchants = pd.read_csv('data/merchants.csv')
transactions = pd.read_csv('data/transactions.csv')

O ficheiro **cities.csv**, que contém informação sobre várias cidades, é constituído por 5 linhas e 5 colunas.

In [ ]:
cities.info()

Cada linha representa uma cidade e as colunas são as seguintes:

0. **city:** nome da cidade
1. **lat:** latitude da cidade
2. **lon:** longitude da cidade
3. **city_pop:** população da cidade
4. **state:** estado a que pertence a cidade

In [ ]:
cities.head()

O ficheiro **customers.csv**, que contém a informação demográfica de cada cliente, é constituído por 1100 linhas e 9 colunas.

In [ ]:
customers.info()

Cada linha representa um cliente e as colunas são as seguintes:

0. **cc_num:** número do cartão de crédito do cliente
1. **first:** primeiro nome do cliente
2. **last:** último nome do cliente
3. **gender:** género do cliente
4. **street:** rua da morada do cliente
5. **city:** cidade da morada do cliente
6. **zip:** código-postal da morada do cliente
7. **job:** emprego do cliente
8. **dob:** data de nascimento do cliente

In [ ]:
customers.head()

O ficheiro **merchants.csv**, que contém os detalhes de cada comerciante, é constituído por 100 linhas e 5 colunas.

In [ ]:
merchants.info()

Cada linha representa um comerciante e as colunas são as seguintes:

0. **merchant:** nome do comerciante
1. **category:** categoria do negócio do comerciante
2. **merch_lat:** latitude do comerciante
3. **merch_lon:** longitude do comerciante
4. **merchant_id:** identificador único do comerciante

In [ ]:
merchants.head()

O ficheiro **transactions.csv**, que contém os registos de transações individuais, é constituído por 30000 linhas e 9 colunas.

In [ ]:
transactions.info()

Cada linha representa uma transação e as colunas são as seguintes:

0. **index:** índice da transação
1. **trans_date_trans_time:** data e hora da transação
2. **cc_num:** número do cartão de crédito associado à transação
3. **device_os:** sistema operativo a partir do qual foi efetuada a transação
4. **merchant:** nome do comerciante da transação
5. **amt:** quantia da transação
6. **trans_num:** número único da transação
7. **unix_time:** *timestamp* *Unix* da transação
8. **is_fraud:** etiqueta que indica se a transação foi fraudulenta (1) ou não (0)

In [ ]:
transactions.head()

Os atributos *trans_date_trans_time*/*unix_time* e *amt* parecem ser aqueles mais relevantes para determinar o valor da variável-alvo (*is_fraud*), por serem os que mais se relacionam com a transação propriamente dita.

Em sentido contrário, as colunas *index*, *trans_num*, *merchant_id*, *first* e *last* deverão ser completamente irrelevantes para o modelo, por serem meramente indicativas. Por isso, estas colunas poderão vir a ser eliminadas numa próxima fase.

Tendo em conta que existe um número bastante elevado de dados (30000 transações, que serão divididas em conjuntos de treino e de teste) e de informação adicional (sobre cidades, clientes e comerciantes), é expectável que se consigam extrair conclusões pertinentes a partir dos dados, isto é, fazer previsões/classificações acertadas, pelo que não se afigura necessário procurar novos dados.

Para além disto, o número de atributos considera-se mais do que suficiente para o problema a tratar, pelo que a dimensionalidade poderá ter de ser reduzida de maneira a evitar a "maldição da dimensionalidade".

Sendo os dados provenientes de uma só origem, não se antecipam quaisquer problemas de compatibilidade/coerência.

Por último, os casos de valores em falta e/ou incorretos serão analisados individualmente: se possível, os valores serão corrigidos ou imputados; senão, as linhas serão eliminadas, visto que o número total de dados é suficiente para a criação de um modelo robusto.

### Descrição dos Dados

A quantidade de dados disponíveis (30000 transações com a respetiva informação) parece adequada para o modelo pretendido, permitindo o desenvolvimento de um modelo simultaneamente eficaz (em termos de métricas de avaliação) e eficiente (em termos de tempo de processamento computacional).

A eliminação de colunas/atributos irrelevantes deverá ser suficiente para escapar à "maldição da dimensionalidade" e evitar *overfitting*, bem como melhorar o desempenho computacional.

Os tipos dos valores e as respetivas escalas apresentam-se abaixo.

**Cidades:** 5 linhas e 5 colunas

| Atributo | Tipo | Escala | Formato |
| -------- | ---- | ------ | ------- |
| **city** | categórico | nominal | *string* |
| **lat** | numérico contínuo | rácio | número decimal |
| **long** | numérico contínuo | rácio | número decimal |
| **city_pop** | numérico discreto | rácio | número inteiro |
| **state** | categórico | nominal | *string* |

Portanto, todos os dados de cidades estão representados de forma adequada.

In [ ]:
cities.dtypes

**Clientes:** 1100 linhas e 9 colunas

| Atributo | Tipo | Escala | Formato |
| -------- | ---- | ------ | ------- |
| **cc_num** | categórico | nominal | número inteiro |
| **first** | categórico | nominal | *string* |
| **last** | categórico | nominal | *string* |
| **gender** | categórico | nominal | *string* |
| **street** | categórico | nominal | *string* |
| **city** | categórico | nominal | *string* |
| **zip** | categórico | nominal | número decimal |
| **job** | categórico | nominal | *string* |
| **dob** | categórico | ordinal | *string* |

Neste caso, os atributos *cc_num* e *zip* são valores categóricos com escala nominal - por não representarem quantidades e não haver uma ordem entre eles -, mas estão representados como números (inteiros e decimais, respetivamente), pelo que não se encontram codificados da forma mais apropriada.

Como tal, caso se pretenda a sua ingestão por parte do modelo, os formatos de representação das colunas *cc_num* e *zip* terão de ser codificados.

In [ ]:
customers.dtypes

**Comerciantes:** 100 linhas e 5 colunas

| Atributo | Tipo | Escala | Formato |
| -------- | ---- | ------ | ------- |
| **merchant** | categórico | nominal | *string* |
| **category** | categórico | nominal | *string* |
| **merch_lat** | numérico contínuo | rácio | número decimal |
| **merch_long** | numérico contínuo | rácio | número decimal |
| **merchant_id** | categórico | nominal | número inteiro |

Aqui, todos os atributos estão representados adequadamente, com exceção de *merchant_id*, que está codificado como um número inteiro, mas é um valor categórico nominal, dado que, apesar de ser representado por um número, corresponde a um identificador que não traduz nenhuma relação de ordem significativa.

Deste modo, o formato da coluna *merchant_id* terá de, posteriormente, ser codificado, se for pretendida a sua ingestão por parte do modelo.

In [ ]:
merchants.dtypes

**Transações:** 30000 linhas e 9 colunas

| Atributo | Tipo | Escala | Formato |
| -------- | ---- | ------ | ------- |
| **index** | categórico | ordinal | número inteiro |
| **trans_date_trans_time** | categórico | ordinal | *string* |
| **cc_num** | categórico | nominal | número inteiro | 
| **device_os** | categórico | nominal | *string* |
| **merchant** | categórico | nominal | *string* |
| **amt** | numérico contínuo | rácio | número decimal |
| **trans_num** | categórico | nominal | *string* |
| **unix_time** | numérico discreto | intervalar | número inteiro |
| **is_fraud** | categórico | nominal | número inteiro |

Neste conjunto de transações, as colunas *index*, *cc_num* e *is_fraud* deveriam ter um formato mais adequado ao seu significado.

Ou seja, tal como anteriormente, estas colunas são do tipo categórico com escala nominal porque, embora os seus valores sejam números inteiros, a ordem entre eles é arbitrária, pelo que, numa fase mais avançada do processo de desenvolvimento, os formatos poderão ter de ser codificados para garantir um tratamento adequado por parte do modelo.

Note-se ainda que a coluna *trans_date_trans_time* é a única com formato *string* que representa valores categóricos ordinais (em vez de nominais), o que poderia levar à necessidade de alguma atenção redobrada num momento posterior, mas, na verdade, esta coluna deverá ser removida por traduzir a mesma informação que a coluna *unix_time* que, por sua vez, tem uma representação mais adequada.

In [ ]:
transactions.dtypes

Em todo o *dataset*, a única coluna que se pode considerar codificada é *is_fraud*, sendo que o esquema de codificação consiste em representar "verdadeiro"/"sim" como 1 e "falso"/"não" como 0. Nenhuma das restantes colunas está codificada.

Compreendidos os tipos e as escalas dos dados, podem compilar-se algumas estatísticas descritivas sobre os mesmos - como a contagem, a unicidade, a frequência, a média, o desvio padrão, os extremos (mínimo e máximo) e os quartis (incluindo a mediana) dos valores presentes em cada coluna -, tendo em vista a constatação de factos e a extração de conclusões.

No conjunto de cidades, pode questionar-se:
1. **"Quantas cidades distintas existem e onde se situam?"**
2. **"Existe algum *outlier* na população das cidades?"**

In [ ]:
cities.describe(include = 'all')

Do análise estatística descritiva do conjunto de cidades, observa-se que:

1. Existem apenas 5 cidades distintas, todas nos Estados Unidos da América;
2. A cidade mais populosa tem mais de 8 milhões de habitantes, estando este valor extremamente distante dos restantes para a mesma coluna.

Relativamente ao conjunto de clientes, pode inquirir-se:
1. **"Qual é a proporção de clientes homens e mulheres?"**
2. **"Qual é o mapeamento das cidades dos clientes no conjunto de cidades?"**
3. **"Quantas profissões distintas têm os clientes?"**

In [ ]:
customers.describe(exclude = 'number')

Analisando estatisticamente os clientes, verifica-se que:

1. A proporção de clientes homens/mulheres é razoavelmente equilibrada (41%/59%);
2. Como existem 6 cidades presentes no conjunto de clientes, pelo menos uma delas não estará mapeada no conjunto de cidades (por serem 5), pelo que não terá informações/detalhes adicionais;
3. Existem apenas 7 profissões distintas de clientes.

As perguntas relevantes para o conjunto de comerciantes são:
1. **"Quantas categorias distintas existem?"**
2. **"Onde se localizam os comerciantes?"**

In [ ]:
merchants.describe(exclude = 'int64')

Com as estatísticas sobre os comerciantes, constata-se que:

1. Existem apenas 5 categorias distintas de comerciantes;
2. O espaço geográfico no qual se encontram os comerciantes é muito mais abrangente do que o espaço geográfico das cidades, isto é, há muitos comerciantes que - pelas suas coordenadas (latitude e longitude) - não se situam nos Estados Unidos da América.

Por último, podem colocar-se questões pertinentes sobre as transações, nomeadamente:
1. **"Existem transações efetuadas ao mesmo tempo?"**
2. **"Qual é o mapeamento dos comerciantes das transações no conjunto de comerciantes?"**
3. **"Existem transações provavelmente duplicadas?"**
4. **"Qual é a proporção de transações fraudulentas e legítimas?"**

In [ ]:
transactions_columns = set(transactions.columns) - {'index', 'cc_num', 'unix_time'}
transactions[list(transactions_columns)].describe(include = 'all')

Através destas estatísticas relativas às transações, evidencia-se que:

1. Existem 2 transações efetuadas exatamente ao mesmo tempo (com ano, dia, hora, minuto e segundo coincidentes), que poderão ser a mesma;
2. Como existem 101 comerciantes presentes no conjunto de transações, pelo menos um deles não estará mapeado no conjunto de comerciantes (por serem 100), pelo que não terá informações/detalhes adicionais;
3. Existem várias transações com o mesmo número (*trans_num*), que deverão ser duplicadas;
4. No conjunto de dados considerado, apenas 2% das transações são fraudulentas.

Assim, é possível concluir que:

- O conjunto de dados é desequilibrado, visto que a variável-alvo (*is_fraud*) não divide as transações em dois grupos com aproximadamente o mesmo tamanho, mas sim num grupo com cerca de 600 transações (fraudulentas) e noutro com cerca de 294000 transações (legítimas), ou seja, quase 50 vezes maior;
- Para além das hipóteses levantadas anteriormente, talvez exista alguma relação entre a legitimidade de uma transação e a localização geográfica do comerciante, isto é, uma transação poderá ser fraudulenta com maior probabilidade se o comerciante se situar fora dos Estados Unidos da América.

### Exploração dos Dados

Nesta fase, desenham-se os gráficos devidamente contextualizados para os atributos considerados relevantes, tendo em vista a extração de novas conclusões.

In [ ]:
def contextualize(title, xlabel, ylabel):
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

Como serão desenhados alguns mapas, define-se desde já a variável com o planisfério.

In [ ]:
world = gpd.read_file(geodatasets.data.naturalearth.land['url'])

Para o conjunto de cidades, os únicos gráficos que podem ser interessantes visualizar são um mapa com a localização de cada cidade e um diagrama de caixa e bigodes com a população de cada cidade, de modo a verificar a correção dos dados, ou seja, para responder à questão: **"Os dados das cidades estão corretos, nomeadamente a sua localização e população?"**.

In [ ]:
plt.figure(figsize = (20, 5))

sb.boxplot(x = cities['city_pop'])
contextualize('População das Cidades', 'População', '')

geometry = [Point(coordinates) for coordinates in zip(cities['long'], cities['lat'])]
gdf = GeoDataFrame(cities, geometry = geometry)

ax = world.plot(figsize = (20, 10))
ax.set_title('Localização das Cidades')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

gdf.plot(ax = ax, color = 'red', markersize = 100)
for x, y, label in zip(gdf.geometry.x, gdf.geometry.y, gdf['city']):
    ax.text(x, y, label, fontweight = 'bold', rotation = 20)

O único ponto que aparece como *outlier* no diagrama de caixa e bigodes é a população máxima no conjunto de cidades (8419600 habitantes) que, tal como observado anteriormente, está fora do intervalo inter-quartis para os dados desta coluna, mas este valor - relativo à cidade *New York* (como visto inicialmente) - está correto, ou seja, corresponde à realidade, pelo que não deve ser corrigido.

A par disso, tal como é possível verificar através do mapa, as coordenadas geográficas das cidades parecem estar aproximadamente corretas, isto é, situadas nos Estados Unidos da América e, por isso, com correspondência com a realidade.

Dado o conjunto de clientes, pode visualizar-se a sua distribuição por cidade, por profissão e por idade (derivada a partir da data de nascimento), distinguindo entre os dois géneros possíveis, na tentativa de encontrar algum padrão significativo nos dados, ou seja, procurar respostas para as perguntas **"Existe algum padrão na distribuição de clientes por cidade? E por profissão? E por idade?"**.

In [ ]:
customers['age'] = pd.to_datetime('today').year - pd.to_datetime(customers['dob']).dt.year

plt.figure(figsize = (20, 5))

plt.subplot(1, 3, 1)
sb.countplot(customers, x = 'city', hue = 'gender', order = sorted(customers['city'].dropna().unique()))
contextualize('Número de Clientes por Cidade', 'Cidade', 'Número de Clientes')

plt.subplot(1, 3, 2)
sb.countplot(customers, x = 'job', hue = 'gender', order = sorted(customers['job'].dropna().unique()))
contextualize('Número de Clientes por Emprego', 'Emprego', 'Número de Clientes')

plt.subplot(1, 3, 3)
sb.histplot(customers[customers['gender'] == 'M']['age'], label = 'M', bins = 10, kde = True)
sb.histplot(customers[customers['gender'] == 'F']['age'], label = 'F', bins = 10, kde = True)
plt.legend(plt.gca().get_legend_handles_labels()[1])
contextualize('Número de Clientes por Idade', 'Idade', 'Número de Clientes')

Observando os três gráficos acima, a distribuição dos dados não parece ter nenhum aspeto a salientar, destacando-se apenas a cidade *Test City* como sendo aquela que não se encontra no conjunto de cidades com informação detalhada, cuja existência já fora constatada previamente.
Posteriormente, poderá ser relevante explorar com mais detalhe esta cidade para perceber se tem algum significado de relevo.

Pode, ainda, comparar-se a distribuição dos clientes por idade com uma distribuição normal, para perceber quão próximas estão ou quão semelhantes são estas distribuições. Esta análise pode ser efetuada através da visualização da própria distribuição dos clientes por idade, bem como de um gráfico quantil-quantil (Q-Q), para responder à questão **"Quão próxima está a distribuição de clientes por idade de uma distribuição normal?"**.

In [ ]:
plt.figure(figsize = (20, 5))

plt.subplot(1, 2, 1)
sb.histplot(customers['age'], stat = 'density', kde = True)

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, customers['age'].mean(), customers['age'].std())
plt.plot(x, p, 'k')
contextualize('Distribuição de Clientes por Idade', 'Idade', 'Densidade de Clientes')

plt.subplot(1, 2, 2)
stats.probplot(customers['age'], plot = plt)
contextualize('Gráfico Quantil-Quantil para a Distribuição de Clientes por Idade', 'Quantis Teóricos', 'Quantis Amostrais')
plt.show()

A análise dos gráficos permite concluir que a distribuição de clientes por idade não segue uma distribuição normal, nem sequer se aproximando ou sendo semelhante a esta. Isto porque, no gráfico da esquerda, a curva da distribuição não é semelhante à curva de uma distribuição normal e, no gráfico da direita, os pontos apresentados não estão próximos da linha reta correspondente à distribuição normal.

Para além disto, é possível comprovar/fortalecer esta conclusão com recurso à computação de métricas e à realização de testes estatísticos. Assim, considere-se o teste de hipóteses cuja hipótese nula é "A distribuição de clientes por idade segue uma distribuição normal".

In [ ]:
statistic, p_value = shapiro(customers['age'])
print('Shapiro-Wilk - Valor-P: {:.2f}'.format(p_value))

statistic, p_value = normaltest(customers['age'])
print('Agostino-Pearson - Valor-P: {:.2f}'.format(p_value))

print('Assimetria: {:.2f}'.format(skew(customers['age'])))
print('Curtose: {:.2f}'.format(kurtosis(customers['age'], fisher = False)))

Efetivamente, os valores-p obtidos extremamente próximos de 0 - portanto, menores do que 0.01 - permitem refutar a hipótese nula com 99% de confiança, assim como os resultados calculados para a assimetria (diferente de 0) e para a curtose (distante de 3) das idades dos clientes. Todos estes dados complementam e consolidam a conclusão de que a distribuição de clientes por idade não segue nem se aproxima de uma distribuição normal.

Por isso, como a distribuição de clientes por idade não está próxima de uma distribuição normal, se se optar por normalizar este atributo, o método correto não será *Standard Scaling*, mas, provavelmente, *Min-Max Scaling*.

Contudo, como este método de normalização é muito sensível a ruído, convém verificar se existem *outliers* nos valores das idades dos clientes, através de um diagrama de caixa e bigodes que responda à pergunta **"Existem *outliers* nos valores das idades dos clientes?"**.

In [ ]:
plt.figure(figsize = (20, 5))

sb.boxplot(x = customers['age'])
contextualize('Idade dos Clientes', 'Idade', '')

customers = customers.drop('age', axis = 1)

Efetivamente, confirma-se que não existem *outliers* nos valores das idades dos clientes, pelo que, para a normalização deste atributo, pode e deve ser utilizado *Min-Max Scaling*.

Considerando o conjunto de comerciantes, pode ser relevante visualizar a sua distribuição por categoria e a localização no mapa.

Enquanto a distribuição por categoria visa apenas - tal como anteriormente - procurar a existência de algum padrão que se saliente nos dados, a localização geográfica dos comerciantes pode ser importante para perceber se existe alguma relação entre essa informação e a legitimidade das transações efetuadas em cada um deles.

Tendo em conta que, por um lado, todos os clientes (à exceção dos residentes em *Test City*) pertencem a cidades nos Estados Unidos da América, mas, por outro lado, muitos comerciantes não se encontram nos Estados Unidos da América, pode haver alguma relação entre a localização geográfica do comerciante e a legitimidade de uma transação a ele associada, tal como, por exemplo, uma transação ser mais provavelmente fraudulenta se tiver sido efetuada num comerciante fora dos Estados Unidos da América.

Em suma, procura-se solucionar as questões **"Existe algum destaque na distribuição dos comerciantes por categoria?"** e **"Poderá haver alguma importância na localização de um comerciante para a classificação de uma transação como fraudulenta ou legítima?"**.

In [ ]:
plt.figure(figsize = (20, 5))

sb.countplot(merchants, x = 'category', palette = {'Apparel': '#e41a1c', 'Electronics': '#4daf4a', 'Entertainment': '#ff7f00', 'Groceries': '#a65628', 'Travel': '#999999'}, order = sorted(merchants['category'].dropna().unique()))
contextualize('Número de Comerciantes por Categoria', 'Categoria', 'Número de Comerciantes')

geometry = [Point(coordinates) for coordinates in zip(merchants['merch_long'], merchants['merch_lat'])]
gdf = GeoDataFrame(merchants, geometry = geometry)

ax = world.plot(figsize = (20, 10))
ax.set_title('Localização dos Comerciantes')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

gdf.plot(ax = ax, column = 'category', cmap = 'Set1', legend = True, markersize = 100)

Ao observar a distribuição dos comerciantes por categoria, não se destaca nenhum padrão que permita extrair qualquer informação/conclusão.

Contudo, visualizando a localização geográfica dos comerciantes no mapa, observa-se que as coordenadas de muitos comerciantes remetem para lugares que se encontram no oceano, ou seja, em localizações impossíveis para um comerciante.
Assim sendo, assume-se que a localização não é uma informação confiável, pelo que deverá vir a ser descartada posteriormente.

No caso do conjunto de transações, existem vários gráficos que se devem explorar, para tentar encontrar respostas a várias questões.

Em particular, pode procurar-se algum padrão na distribuição de transações fraudulentas por hora, ou por dia da semana, ou por mês - **"Existe algum padrão na distribuição de transações (fraudulentas) por hora? E por dia? E por mês?"**.

In [ ]:
timestamp = pd.to_datetime(transactions['unix_time'], unit = 's')
transactions['hour'] = timestamp.dt.hour
transactions['weekday'] = timestamp.dt.weekday
transactions['month'] = timestamp.dt.month

plt.figure(figsize = (20, 10))

plt.subplot(2, 3, 1)
sb.histplot(transactions[transactions['is_fraud'] == 0]['hour'], label = 'Not Fraud', bins = range(25), kde = True)
sb.histplot(transactions[transactions['is_fraud'] == 1]['hour'], label = 'Fraud', bins = range(25), kde = True)
plt.legend(plt.gca().get_legend_handles_labels()[1])
plt.xticks(range(25))
contextualize('Número de Transações por Hora', 'Hora', 'Número de Transações')

plt.subplot(2, 3, 2)
sb.countplot(transactions, x = 'weekday', hue = 'is_fraud', order = sorted(transactions['weekday'].dropna().unique()))
plt.xticks(ticks = range(7), labels = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado', 'Domingo'])
contextualize('Número de Transações por Dia da Semana', 'Dia da Semana', 'Número de Transações')

plt.subplot(2, 3, 3)
sb.countplot(transactions, x = 'month', hue = 'is_fraud', order = sorted(transactions['month'].dropna().unique()))
plt.xticks(ticks = range(12), labels = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
contextualize('Número de Transações por Mês', 'Mês', 'Número de Transações')

plt.subplot(2, 3, 4)
sb.histplot(transactions[transactions['is_fraud'] == 1]['hour'], bins = range(25), kde = True, color = 'darkorange')
plt.xticks(range(25))
contextualize('Número de Transações Fraudulentas por Hora', 'Hora', 'Número de Transações Fraudulentas')

plt.subplot(2, 3, 5)
sb.countplot(transactions[transactions['is_fraud'] == 1], x = 'weekday', color = 'darkorange', order = sorted(transactions['weekday'].dropna().unique()))
plt.xticks(ticks = range(7), labels = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado', 'Domingo'])
contextualize('Número de Transações Fraudulentas por Dia da Semana', 'Dia da Semana', 'Número de Transações Fraudulentas')

plt.subplot(2, 3, 6)
sb.countplot(transactions[transactions['is_fraud'] == 1], x = 'month', color = 'darkorange', order = sorted(transactions['month'].dropna().unique()))
plt.xticks(ticks = range(12), labels = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
contextualize('Número de Transações Fraudulentas por Mês', 'Mês', 'Número de Transações Fraudulentas')

transactions = transactions.drop(['hour', 'weekday', 'month'], axis = 1)

Efetivamente, desprezando ligeiras flutuações, não se consegue extrair nenhuma conclusão significativa da análise deste gráficos.

Verifica-se uma tendência de existirem alguns intervalos de horas com menos transações fraudulentas, bem como uma menor ocorrência das mesmas ao fim de semana (principalmente ao domingo) e no mês de abril, mas estas informações não parecem ser muito relevantes por não se salientarem de forma extremamente evidente, nem parecer haver uma justificação lógica para as mesmas que não seja serem puramente fruto do acaso.

Ainda assim, a discriminação de cada transação no seu dia da semana e mês pode ser relevante para o modelo extrair conclusões sobre a legitimidade da transação.

Pode ainda ser interessante verificar se existe alguma relação entre a legitimidade de uma transação e o sistema operativo associado ao dispositivo em que foi efetuada, bem como relativamente à quantia das mesmas - **"Será que o sistema operativo pode auxiliar na classificação de uma transação como fraudulenta? E a quantia?"**.

In [ ]:
plt.figure(figsize = (20, 10))

plt.subplot(2, 2, 1)
sb.countplot(transactions, x = 'device_os', hue = 'is_fraud', order = sorted(transactions['device_os'].dropna().unique()))
contextualize('Número de Transações por Sistema Operativo', 'Sistema Operativo', 'Numero de Transações')

plt.subplot(2, 2, 2)
sb.histplot(transactions[transactions['is_fraud'] == 0]['amt'], label = 'Not Fraud', bins = 10, kde = True)
sb.histplot(transactions[transactions['is_fraud'] == 1]['amt'], label = 'Fraud', bins = 10, kde = True)
plt.legend(plt.gca().get_legend_handles_labels()[1])
contextualize('Número de Transações por Quantia', 'Quantia', 'Número de Transações')

plt.subplot(2, 2, 3)
sb.countplot(transactions[transactions['is_fraud'] == 1], x = 'device_os', color = 'darkorange', order = sorted(transactions['device_os'].dropna().unique()))
contextualize('Número de Transações Fraudulentas por Sistema Operativo', 'Sistema Operativo', 'Numero de Transações Fraudulentas')

plt.subplot(2, 2, 4)
sb.histplot(transactions[transactions['is_fraud'] == 1]['amt'], bins = 10, kde = True, color = 'darkorange')
contextualize('Número de Transações Fraudulentas por Quantia', 'Quantia', 'Numero de Transações Fraudulentas')

Contudo, ao observar os gráficos desenhados, também não parece haver nenhuma relação muito evidente entre o sistema operativo e a legitimidade de uma transação.

Quanto à quantia, a única conclusão que se pode extrair é que parece existir uma tendência para as transações fraudulentas estarem associadas a montantes medianos (entre os 100 e os 400), isto é, nem extremamente baixos (< 100), nem extremamente altos (> 400).

A explicação para isto pode residir em dois fatores. Por um lado, na perspetiva de um atacante, uma transação fraudulenta de quantia reduzida pode não compensar/justificar o risco de ser apanhado. Por outro lado, admitindo que uma transação de quantia elevada é mais escrutinada, um atacante pode também considerar que tentar uma transação fraudulenta acima de um dado valor será demasiado arriscado e aumentará a probabilidade de ser detetado. Por isso, poderá haver uma tendência para as transações fraudulentas estarem associadas a montantes suficientemente elevados para justificarem o risco que lhes é inerente, mas suficientemente reduzidos para não serem demasiado arriscadas.

Para além disto, é também possível verificar a distribuição das transações por quantia de maneira a compará-la com uma distribuição normal para compreender se existe alguma semelhança ou aproximação entre estas duas distribuições. Isto pode ser analisado por meio da visualização da distribuição das transações por quantia e de um gráfico quantil-quantil (Q-Q), na tentativa de solucionar a questão **"A distribuição de transações por quantia pode ser considerada normal?"**.

In [ ]:
plt.figure(figsize = (20, 5))

plt.subplot(1, 2, 1)
sb.histplot(transactions['amt'], stat = 'density', kde = True)

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, transactions['amt'].mean(), transactions['amt'].std())
plt.plot(x, p, 'k')
contextualize('Distribuição de Transações por Quantia', 'Quantia', 'Densidade de Transações')

plt.subplot(1, 2, 2)
stats.probplot(transactions['amt'].dropna(), plot = plt)
contextualize('Gráfico Quantil-Quantil para a Distribuição de Transações por Quantia', 'Quantis Teóricos', 'Quantis Amostrais')
plt.show()

Ambos os gráficos evidenciam que a distribuição de transações por quantia não está próxima de uma distribuição normal, visto que a curva da distribuição é bastante diferente da curva correspondente a uma distribuição normal e o traçado no gráfico quantil-quantil também não se aproxima de uma linha reta, tal como seria o caso numa distribuição normal.

A par disto, este facto pode ser consolidado através de algumas métricas e do resultado de testes estatísticos. Tal como anteriormente, a hipótese nula dos testes de hipóteses é "A distribuição de transações por quantia segue uma distribuição normal".

In [ ]:
amt = transactions['amt'].dropna()

statistic, p_value = shapiro(amt)
print('Shapiro-Wilk - Valor-P: {:.2f}'.format(p_value))

statistic, p_value = normaltest(amt)
print('Agostino-Pearson - Valor-P: {:.2f}'.format(p_value))

print('Assimetria: {:.2f}'.format(skew(amt)))
print('Curtose: {:.2f}'.format(kurtosis(amt, fisher = False)))

Deste modo, como os valores-p resultantes dos testes de hipóteses estão extremamente perto de 0, sendo, portanto, menores do que 0.01, é possível refutar a hipótese nula com uma confiança de 99%. Além disto, o valor obtido para a curtose (diferente de 3) sustenta a conclusão de que a distribuição de transações por quantia não é uma distribuição normal.

Como tal, estando garantido que a distribuição de transações por quantia não segue nem se aproxima de uma distribuição normal, caso seja necessário normalizar esta variável, a abordagem correta é utilizar *Min-Max Scaling*, desde que não existam *outliers* nos dados desta coluna.

Assim, a ausência de *outliers* deve ser confirmada através de um diagrama de caixa e bigodes que solucione a questão **"Existem *outliers* nas quantias das transações?"**.

In [ ]:
plt.figure(figsize = (20, 5))

sb.boxplot(x = transactions['amt'])
contextualize('Quantia das Transações', 'Quantia', '')

De facto, não se verifica a existência de nenhum valor *outlier* nas quantias da transações, de maneira que o recurso a *Min-Max Scaling* para a normalização desta coluna é válido e está justificado.

Pode também ser relevante procurar alguma ligação entre as transações fraudulentas e determinados comerciantes - **"Há alguma ligação entre a legitimidade de uma transação e o comerciante em que é efetuada?"**.

In [ ]:
plt.figure(figsize = (5, 25))

table = pd.crosstab(transactions['merchant'], transactions['is_fraud'])
table = table.sort_values(by = 'merchant', key = lambda x: x.str.split('_').str[1].apply(lambda val: int(val) if val.isdigit() else float('inf')))
row_sum = table.sum(axis = 1)
percentages = (table.T / row_sum * 100).T
annotations = [['{} ({}%)'.format(count, round(percentage)) for percentage, count in zip(row, table_row)] for table_row, row in zip(table.values, percentages.values)]
sb.heatmap(percentages, annot = annotations, fmt = '', cmap = 'coolwarm')
contextualize('Ilegitimidade das Transações por Comerciante', 'Ilegitimidade das Transações', 'Comerciante')

Ao analisar o gráfico acima, podem extrair-se duas conclusões.

Em primeiro lugar, o comerciante *Unknown_Merchant* é aquele - cuja existência já fora identificada anteriormente - que não se encontra mapeado no conjunto de comerciantes com informação detalhada.
Em segundo lugar, observa-se também que este é o único comerciante que não tem nenhuma transação fraudulenta associada, sendo legítimas todas as transações nele efetuadas.

Para além disto, não se observa mais nenhuma relação significativa entre transações fraudulentas e determinados comerciantes, tendo todos um rácio de transações fraudulentas que se encontra entre 1% e 4%.

Finalmente, pode existir alguma correlação significativa entre as variáveis, identificável através de um correlograma - **"Algum atributo está fortemente correlacionado com a legitimidade de uma transação?"**.

In [ ]:
correlation = transactions.select_dtypes(['float', 'int']).corr()

plt.figure(figsize = (5, 5))
sb.heatmap(correlation, annot = True, cmap = 'coolwarm', mask = np.triu(np.ones_like(correlation, dtype = bool)), vmin = -1, vmax = 1)
contextualize('Correlograma de Transações', '', '')

No correlograma, existem apenas duas células que se destacam visualmente: (*unix_time*, *index*) com um coeficiente de correlação de 0,66 e (*is_fraud*, *index*) com um coeficiente de correlação de -0,1.

Contudo, como o atributo *index* é simplesmente o índice da transação, sendo, por isso, meramente indicativo e não informativo, não se pode extrair nada de significativo destas correlações. Efetivamente, o atributo *index* será, numa fase posterior, removido do conjunto de dados.

Assim, não se identifica nenhuma correlação relevante entre os atributos analisados para o conjunto de transações.

Em suma, extraem-se as seguintes conclusões desta fase:

- O conjunto de cidades parece conter dados corretos e plausíveis;
- No conjunto de clientes, o único fator de relevo é a existência de uma cidade *Test City*, que não se encontra mapeada no conjunto de cidades;
- No conjunto de comerciantes, a localização geográfica não parece ser uma informação credível, por remeter para demasiados pontos inviáveis/impossíveis, pelo que deverá ser desconsiderada;
- No conjunto de transações:
  - O único padrão/tendência que pode ter algum significado relevante é o facto de haver mais transações fraudulentas com quantias entre os 100 e os 400, em comparação com as transações de montantes fora deste intervalo;
  - Existe um comerciante *Unknown_Merchant* que é o único que só tem associadas transações legítimas;
  - Não existem correlações evidentes de alguma variável com a variável-alvo (*is_fraud*).

### Verificação da Qualidade dos Dados

Antes de preparar os dados para a sua ingestão por parte do modelo, deve aferir-se a sua qualidade, em particular quanto a valores em falta, valores duplicados/redundantes, erros (de codificação ou de medição), ou inconsistências, entre outros.

Note-se que as funções `pandas.DataFrame.isna()` e `pandas.DataFrame.isnull()` são estritamente equivalentes, isto é, têm exatamente o mesmo significado e realizam exatamente o mesmo, pelo que só se utilizará uma delas.

In [ ]:
cities.isnull().sum()

Deste modo, verifica-se que não existem valores em falta no conjunto de cidades.

In [ ]:
customers.isnull().sum()

No conjunto de clientes, existem 20 clientes cujo código-postal e/ou profissão estão em falta, pelo que esses casos devem ser observados de forma mais detalhada.

In [ ]:
customers[customers.isnull().any(axis = 1)]

Tal como é possível observar, todos os clientes com código-postal em falta também têm a profissão em falta e vice-versa. Ora, não sendo recomendável eliminar clientes por serem uma parte fundamental da transação, a decisão a tomar deve ser ponderada.

O atributo *zip* - categórico nominal - será posteriormente removido, dado que, para além de não demonstrar ser relevante para a decisão da variável-alvo, a sua codificação resultaria num aumento substancial do número de colunas a tratar pelo modelo, correndo o risco de degradar substancialmente o seu desempenho.

Quanto ao atributo *job*, os valores em falta serão imputados com o valor mais comum para o emprego de clientes com o mesmo género e da mesma cidade, de maneira a tentar preservar tanto quanto possível as propriedades do conjunto inicial de dados.

In [ ]:
merchants.isnull().sum()

No conjunto de comerciantes, existem apenas 2 comerciantes com valores em falta (categoria e latitude), pelo que serão analisados.

In [ ]:
merchants[merchants.isnull().any(axis = 1)]

De forma análoga à anterior, os comerciantes com a categoria em falta também têm a latitude em falta e vice-versa. Ora, tal como anteriormente, estas linhas não devem ser eliminadas, visto que os comerciantes são elementos essenciais de cada transação, pelo que se deve deliberar a ação a efetuar.

A coluna *merch_lat* já foi determinada como não confiável, o que leva a que os valores em falta nesta coluna não sejam problemáticos e não tenham necessidade de ser tratados, dado que a coluna será, posteriormente, eliminada.

A coluna *category* pode ser relevante para o modelo, de modo que os valores em falta devem ser imputados. Dada a escassez de atributos válidos para os comerciantes, o que inviabiliza uma imputação considerando os valores mais próximos/semelhantes, a imputação da categoria será feita com a categoria mais comum (a moda), à falta de melhor opção.

In [ ]:
transactions.isnull().sum()

Finalmente, o conjunto de transações é aquele que tem mais valores em falta e - dada a sua maior importância - deve ser alvo de uma análise mais aprofundada.

Os valores em falta na coluna *trans_date_trans_time* não são relevantes, porque esta coluna traduz a mesma informação que a coluna *unix_time* (é redundante), pelo que o atributo *trans_date_trans_time* será removido posteriormente, sem perda de informação.

In [ ]:
transactions[transactions['amt'].isnull()][transactions['is_fraud'] == 1]

Dos 100 valores em falta da coluna *amt*, apenas 2 correspondem a transações fraudulentas.

Uma destas duas transações tem em falta, para além da quantia, o valor do atributo *device_os*, pelo que a sua eliminação é a decisão mais adequada a tomar dado que, caso contrário, teriam de ser imputados valores em duas colunas.

In [ ]:
transactions[transactions['trans_num'] == 'TRANS_471519']

A outra transação com quantia em falta parece ser um erro, dado que existe uma outra transação com o mesmo número (que devia ser único), mas com dados totalmente diferentes. Como tal, para evitar introduzir erros no modelo, ambas as transações com o número *TRANS_471519* deverão ser eliminadas futuramente.

Assim, todas as 100 transações com o valor do atributo *amt* em falta podem ser eliminadas com segurança, sem correr o risco de perder muita informação relevante para o processo de decisão.

Por último, note-se que existem quase 18000 valores em falta na coluna *device_os*, o que corresponde a mais de metade do conjunto de transações (30000). Assim sendo, visto que (1) eliminar as linhas com valores em falta levaria a uma redução drástica do conjunto de dados para menos de metade do conjunto inicial e (2) este atributo não se mostrou ser relevante para o modelo, a decisão a tomar é remover a coluna *device_os*.

Concluída a análise sobre valores em falta, devem tambem procurar-se valores duplicados/redundantes e eventuais erros.

In [ ]:
cities.nunique()

No conjunto de cidades, todos os valores são únicos e a sua correção pode ser verificada através da observação de todos os valores, tendo em conta que são apenas 5.

In [ ]:
cities

Observando todos os valores presentes no conjunto de cidades e comparando-os com os dados reais, confirma-se que todos os valores estão corretos.

In [ ]:
cities.duplicated().sum()

Efetivamente, constata-se que não existem cidades duplicadas.

In [ ]:
cities['city'].duplicated().sum()

Em particular, verifica-se também que não existem nomes de cidades duplicados, tal como esperado, tendo em conta que esta coluna deve corresponder à chave primária da tabela/relação.

In [ ]:
customers.nunique()

No conjunto de clientes, verifica-se que apenas existem 108 nomes e 108 apelidos, o que não parece plausível para 1100 pessoas. Isto poderá significar que os nomes e apelidos foram modificados para garantir o anonimato dos clientes, o que dá mais força à teoria de serem atributos irrelevantes para o modelo, pelo que podem e devem ser removidos posteriormente.

De forma análoga, 102 moradas distintas parece um valor extremamente pequeno para 1100 pessoas, o que deverá ser um indicativo do mesmo facto, isto é, de que as moradas foram alteradas para assegurar o anonimato dos clientes e, por isso, este é mais um atributo sem relevância para o modelo, pelo que deve ser eliminado posteriormente.

Em sentido inverso, o número de códigos-postais únicos e de datas de nascimento únicas parece plausível, ainda que isso não signifique que são atributos relevantes, mas apenas que os seus valores deverão estar corretos.

In [ ]:
customers['gender'].unique()

Os valores para o género também estão corretos, de acordo com a representação esperada (*M* para masculino e *F* para feminino).

In [ ]:
customers['city'].unique()

Na coluna *city*, tal como verificado anteriormente no gráfico "Numero de Clientes por Cidade", destaca-se apenas a cidade *Test City*.
Continuam a subsistir dúvidas sobre o significado desta cidade, mas, apesar de não pertencer ao conjunto de cidades, não pode ser considerada um erro, pelo que não se planeia, para já, a sua remoção do conjunto de dados.

In [ ]:
customers['job'].unique()

O atributo *job* contém os valores únicos esperados, de acordo com o observado anteriormente no gráfico "Número de Clientes por Emprego". Os clientes com o valor do emprego como *nan* terão este valor imputado, em conformidade com o que já fora decidido previamente.

In [ ]:
customers.duplicated().sum()

Ao procurar por clientes duplicados, não se encontra nenhum, o que parece indicar que neste conjunto não existem valores duplicados.

In [ ]:
customers['cc_num'].duplicated().sum()

De facto, confirma-se que não existem dois clientes com o mesmo número de cartão de crédito (o que seria impossível, logo, seriam duplicados), pelo que não existem duplicados no conjunto de clientes.

In [ ]:
merchants.nunique()

A análise da unicidade nos atributos do conjunto de comerciantes revela que todas as colunas têm o número esperado de valores únicos/distintos, notando que a diferença na coluna *merch_lat* (98) em relação às restantes (100) se deve à existência de dois comerciantes com o valor deste atributo em falta. No entanto, este atributo será removido, tal como já discutido previamente.

In [ ]:
merchants['category'].unique()

Analogamente aos empregos dos clientes, também as categorias dos comerciantes vão ao encontro do esperado para os valores desta coluna (já visualizados no gráfico "Número de Comerciantes por Categoria"), sendo que os comerciantes sem categoria (*nan*) terão este valor imputado com a moda, pelas razões já explicitadas.

In [ ]:
merchants.duplicated().sum()

Mais uma vez, não parecem existir comerciantes duplicados.

In [ ]:
merchants['merchant'].duplicated().sum()

In [ ]:
merchants['merchant_id'].duplicated().sum()

Efetivamente, nenhuma das chaves candidatas (*merchant* e *merchant_id*) para a tabela/relação de comerciantes tem valores duplicados, pelo que está garantida a ausência de duplicados no conjunto de comerciantes.

In [ ]:
transactions.nunique()

Por fim, os valores únicos nas colunas do conjunto de transações transmitem algumas informações relevantes.

Os valores únicos para os atributos *trans_date_trans_time* e *unix_time* não evidenciam nenhum problema com os dados, visto que é possível que ocorram duas transações diferentes no mesmo instante.

A par disto, a unicidade nas colunas *cc_num* e *amt* também não revela nenhum padrão potencialmente anómalo ou suspeito, sendo estes valores plausíveis para um conjunto de 30000 transações (cerca de 27 transações por cliente/cartão de crédito, com 22615 quantias/montantes diferentes).

In [ ]:
transactions['device_os'].unique()

Os valores únicos na coluna *device_os* correspondem aos já observados anteriormente no gráfico "Número de Transações por Sistema Operativo", sendo que já foi decidido o tratamento para este atributo - eliminação.

In [ ]:
transactions['merchant'].unique()

Igualmente, os comerciantes associados às transações, pelo seu número (101), vão ao encontro do esperado - considerando o gráfico "Número de Transações por Comerciante" -, ressalvando a existência de um *Unknown_Merchant* que não consta do conjunto de comerciantes e no qual todas as transações são legítimas.

In [ ]:
transactions[transactions['merchant'] == 'Unknown_Merchant']

Observando com mais detalhe as transações associadas ao comerciante *Unknown_Merchant*, verifica-se que, para além de serem todas legítimas, todas estão associadas ao número de cartão de crédito *9999999999999999*, o que parece indicar um erro nestes registos. Como tal, tomando em consideração que, dado o elevado de número de transações legítimas, eliminar 10 transações deste tipo não impactará significativamente o modelo, assume-se que a decisão mais acertada é eliminar estes registos.

In [ ]:
transactions['is_fraud'].unique()

Os valores na coluna *is_fraud* (variável-alvo) seguem a codificação esperada, isto é, 0 para transação legítima e 1 para transação fraudulenta.

Concluída a análise destas colunas, restam os atributos *index* e *trans_num*, que deveriam ser únicos, isto é, deveriam existir tantos valores nestas colunas como o número de transações, ou seja, 30000. Contudo, não é isto que sucede.

Em primeiro lugar, como o número de valores únicos na coluna *index* (29970) é inferior ao número de transações (30000), deverão existir valores duplicados nesta coluna.

In [ ]:
transactions['index'].duplicated().sum()

Ora, como existem valores duplicados para o atributo *index* e tal não devia acontecer, as linhas duplicadas terão de ser removidas posteriormente.

In [ ]:
transactions[transactions['index'].duplicated()][transactions['is_fraud'] == 1]

Contudo, nenhuma das linhas com índice duplicado corresponde a uma transação fraudulenta, o que permite eliminar estas transações sem perder informação muito relevante para o modelo, dada a escassez de transações fraudulentas, em contraste com a abundância de transações legítimas.

Em segundo lugar, similarmente, o valor do atributo *trans_num* deveria ser único para cada transação, mas existem apenas 29470 valores distintos nesta coluna, quando deveriam ser tantos quantas as transações, isto é, 30000. Como tal, existirão duplicados nesta coluna, o que compromete a integridade das transações a que estão associados.

In [ ]:
transactions['trans_num'].duplicated().sum()

Efetivamente, confirma-se a existência de 530 transações com o número duplicado, pelo que terão de ser eliminadas futuramente.

In [ ]:
transactions[transactions['trans_num'].duplicated()][transactions['is_fraud'] == 1]

Observando de forma detalhada as transações com *trans_num* duplicado, conclui-se que apenas 6 das 530 (1,1%) são fraudulentas, o que permite inferir que a eliminação das 530 transações com número duplicado não prejudicará o modelo, dado que apenas é removida uma pequena fração das transações fraudulentas e que, de outro modo, seria impossível garantir a integridade do conjunto de transações.

In [ ]:
transactions.duplicated().sum()

Assim, serão, consequentemente, eliminadas as 9 transações duplicadas.

Numa análise agregada dos quatro conjuntos de dados, tomando em consideração as conclusões observadas nesta fase e na anterior, constata-se que:
- Todos os valores do conjunto de cidades parecem estar corretos;
- Excluindo as colunas *first*, *last* e *street*, todos os valores do conjunto de clientes parecem plausíveis;
- No conjunto de comerciantes, as coordenadas de localização geográfica (*merch_lat* e *merch_long*) devem ser eliminadas por não serem confiáveis;
- O conjunto de transações tem mais especificidades, detalhadas de seguida.

Em suma, as conclusões que se extraem desta etapa para fase de preparação dos dados são as seguintes:
- O conjunto de cidades não deve sofrer alterações (devidas à falta de qualidade dos dados, podendo sofrer alterações por outros motivos, nomeadamente pela relevância dos seus atributos);
- No conjunto de clientes:
  - Eliminar/Remover:
    - As colunas *first*, *last* e *street*, por estarem alteradas/modificadas para garantir anonimato e serem irrelevantes;
    - A coluna *zip*, por ter valores em falta, ser provavelmente irrelevante e piorar o desempenho do modelo devido à necessidade de codificação;
  - Imputar:
    - O valor do atributo *job* com o emprego mais comum para o género e a cidade do cliente, quando estiver em falta (20);
- No conjunto de comerciantes:
  - Eliminar/Remover:
    - As colunas *merch_lat* e *merch_long*, por não serem confiáveis;
  - Imputar:
    - O valor do atributo *category* com a moda das categorias, quando estiver em falta (2);
- No conjunto de transações:
  - Eliminar/Remover:
    - A coluna *trans_date_trans_time*, por ser redundante (em relação a *unix_time*);
    - A coluna *device_os*, por ter mais de metade dos valores em falta e aparentar ser irrelevante;
    - As linhas com o atributo *amt* em falta (100);
    - A linha com o valor do atributo *trans_num* a *TRANS_471519*;
    - As linhas com o valor do atributo *cc_num* a *9999999999999999* e *merchant* a *Unkown_Merchant* (10);
    - As linhas com o valor do atributo *index* duplicado (30);
    - As linhas com o valor do atributo *trans_num* duplicado (530);
    - As linhas duplicadas (9), como consequência das eliminações/remoções anteriores.

Assim, globalmente, foram identificados os erros do conjunto de dados e decididas quais as medidas a tomar para os corrigir.

## Preparação dos Dados

### Seleção dos Dados

No conjunto de cidades, toda a informação é considerada correta. Contudo, dado que a localização dos comerciantes foi considerada incorreta e, por isso, será removida, a localização das cidades vê a sua relevância diminuída, pelo que deverá, igualmente, ser excluída.

Assim, restam os atributos *city*, *city_pop* e *state*.

Numa fase posterior, analisada a relevância destes atributos, poderão vir a ser removidos. Ainda assim, nesta fase, serão mantidos no conjunto de dados.

In [ ]:
selected_cities = cities.drop(['lat', 'long'], axis = 1)

selected_cities.head()

Do conjunto de clientes, foi previamente determinado que as colunas *first*, *last* e *street* foram alteradas/modificadas para garantir o anonimato dos clientes, tornando-se, assim, irrelevantes, pelo que esses atributos devem ser removidos. A par disso, também foi decidida a eliminação da coluna *zip*, considerando o número de valores em falta, a ausência de relevância para o modelo e o impacto significativo no desempenho.

Os restantes atributos podem ser mantidos para já, mas a sua relevância será determinada posteriormente, verificando-se a existência de alguma relação com a variável-alvo.

In [ ]:
selected_customers = customers.drop(['first', 'last', 'street', 'zip'], axis = 1)

selected_customers.head()

Do conjunto de comerciantes, observou-se que as coordenadas de localização geográfica não são confiáveis, pelo que os atributos *merch_lat* e *merch_long* devem ser eliminados.

Para além disto, o atributo *merchant* é aquele que será útil para a junção desta tabela com o conjunto de transações, de maneira que o atributo *merchant_id* se torna redundante, sendo, assim, eliminado.

In [ ]:
selected_merchants = merchants.drop(['merch_lat', 'merch_long', 'merchant_id'], axis = 1)

selected_merchants.head()

No conjunto de transações, a coluna *trans_date_trans_time* é redundante (devido à existência do atributo *unix_time*), de modo que foi decidido optar-se pela sua eliminação. Para além desta coluna, também foi decidida a eliminação do atributo *device_os*, dada a elevada quantidade de valores em falta e a aparente pouca relevância do sistema operativo na determinação da variável-alvo.

Os atributos *index* e *trans_num*, ainda que sejam irrelevantes, não podem ser removidos nesta fase, porque serão necessários para proceder à limpeza dos dados. Todavia, efetuada a limpeza necessária dos dados, estas colunas serão removidas por serem irrelevantes.

In [ ]:
selected_transactions = transactions.drop(['trans_date_trans_time', 'device_os'], axis = 1)

selected_transactions.head()

### Limpeza dos Dados

Conforme o que foi anteriormente determinado, não existe necessidade de limpar o conjunto de cidades.

In [ ]:
clean_cities = selected_cities

No conjunto de clientes, devem ser imputados os 20 valores de *job* em falta com o valor mais frequente/comum para o género e a cidade de cada cliente.

In [ ]:
def impute_job(customer):
    if pd.isna(customer['job']):
        neighbors = customers[(customers['gender'] == customer['gender']) & (customers['city'] == customer['city'])]
        mode = neighbors['job'].mode()[0]
        return mode
    return customer['job']

clean_customers = selected_customers

before = clean_customers['job'].isna().sum()

clean_customers['job'] = clean_customers.apply(impute_job, axis = 1)

print('Linhas Afetadas:', before - clean_customers['job'].isna().sum())

No conjunto de comerciantes, devem ser imputados os 2 valores de *category* em falta com a categoria mais comum.

In [ ]:
clean_merchants = selected_merchants

before = clean_merchants['category'].isna().sum()

mode = clean_merchants['category'].mode()[0]
clean_merchants['category'] = clean_merchants['category'].fillna(mode)

print('Linhas Afetadas:', before - clean_merchants['category'].isna().sum())

No conjunto de transações, devem ser limpas:
  - As 100 linhas com o atributo *amt* em falta;

In [ ]:
clean_transactions = selected_transactions

before = clean_transactions.shape[0]

clean_transactions = clean_transactions.dropna(subset = ['amt'])

print('Linhas Afetadas:', before - clean_transactions.shape[0])

  - A linha com o valor do atributo *trans_num* a *TRANS_471519*;

In [ ]:
before = clean_transactions.shape[0]

clean_transactions = clean_transactions[clean_transactions['trans_num'] != 'TRANS_471519']

print('Linhas Afetadas:', before - clean_transactions.shape[0])

  - As 10 linhas com o valor do atributo *cc_num* a *9999999999999999* e *merchant* a *Unkown_Merchant*;

In [ ]:
before = clean_transactions.shape[0]

clean_transactions = clean_transactions[(clean_transactions['cc_num'] != 9999999999999999) & (clean_transactions['merchant'] != 'Unknown_Merchant')]

print('Linhas Afetadas:', before - clean_transactions.shape[0])

  - As 30 linhas com o valor do atributo *index* duplicado;

In [ ]:
before = clean_transactions.shape[0]

clean_transactions = clean_transactions.drop_duplicates(subset = ['index'])

print('Linhas Afetadas:', before - clean_transactions.shape[0])

Note-se que o valor de linhas limpas é menor do que 30 porque algumas linhas com o valor do atributo *index* duplicado já terão sido eliminadas fruto de limpezas anteriormente efetuadas.

  - As 530 linhas com o valor do atributo *trans_num* duplicado;

In [ ]:
before = clean_transactions.shape[0]

clean_transactions = clean_transactions.drop_duplicates(subset = ['trans_num'])

print('Linhas Afetadas:', before - clean_transactions.shape[0])

Note-se que o valor de linhas limpas é menor do que 530 porque algumas linhas com o valor do atributo *trans_num* duplicado já terão sido eliminadas fruto de limpezas anteriormente efetuadas.

  - As 9 linhas duplicadas, como consequência das eliminações/remoções anteriores;

In [ ]:
before = clean_transactions.shape[0]

clean_transactions = clean_transactions.drop_duplicates()

print('Linhas Afetadas:', before - clean_transactions.shape[0])

Assim, confirma-se que todas as linhas duplicadas já tinham sido previamente removidas e não restam valores duplicados no conjunto de transações.

Tendo resolvido todos os problemas de valores em falta no conjunto de transações, já podem ser removidas as colunas irrelevantes *index* e *trans_num*.

In [ ]:
clean_transactions = clean_transactions.drop(['index', 'trans_num'], axis = 1)

clean_transactions.head()

Deste modo, dá-se por concluída a limpeza dos dados, tendo sido resolvidos os problemas de valores em falta e os erros nos dados.

### Construção de Novos Dados

De maneira a tentar melhorar o desempenho do modelo a desenvolver, pode ser seguido um processo de engenharia de funcionalidades para construir novos dados, derivados dos atributos existentes.

Neste caso em particular, no conjunto de cidades e de comerciantes, dado o escasso número de atributos e de informação sobre as instâncias, não se identificam nenhumas funcionalidades relevantes a construir.

In [ ]:
new_cities = clean_cities
new_merchants = clean_merchants

No entanto, nos conjuntos de clientes e de transações, existem quatro atributos que se considera que, trabalhados de algum modo, podem trazer informação mais relevante para o modelo.

Em primeiro lugar, no conjunto de clientes, existe o atributo *dob* para representar a data de nascimento de cada cliente. Contudo, tendo em conta que a data de nascimento tem como principal propósito a determinação da idade do cliente, este atributo *dob* pode ser substituído por um novo atributo já com a idade do cliente, denominado *age*, derivado a partir da data de nascimento.

Todavia, como a idade de cada cliente tem relevância no momento de cada transação efetuada (e não no momento atual), este novo atributo só será criado após a integração dos dados, a efetuar na próxima etapa.

In [ ]:
new_customers = clean_customers

Em segundo lugar, no conjunto de transações, o atributo *unix_time* representa o momento em que ocorreu cada transação. Todavia, este atributo pode tornar-se mais significativo e facilmente interpretável pelo modelo se for convertido/transformado numa data da transação com informação sobre o mês e o dia da semana, por terem sido os atributos anteriormente considerados mais relevante, na fase de Exploração dos Dados. No entanto, os valores destes atributos são cíclicos, pelo que se devem codificar utilizando *cyclical encoding* (considerando o círculo trigonométrico), resultando, assim, nas colunas *month_sin*, *month_cos*, *weekday_sin* e *weekday_cos*.

In [ ]:
new_transactions = clean_transactions

timestamp = pd.to_datetime(new_transactions['unix_time'], unit = 's')

month = (timestamp.dt.month - 1) * (2 * np.pi / 12)
new_transactions['month_sin'] = round(np.sin(month), 3)
new_transactions['month_cos'] = round(np.cos(month), 3)

weekday = timestamp.dt.weekday * (2 * np.pi / 7)
new_transactions['weekday_sin'] = round(np.sin(weekday), 3)
new_transactions['weekday_cos'] = round(np.cos(weekday), 3)

new_transactions.head()

Em terceiro lugar, dado o contexto de deteção de fraude em transações de cartão de crédito, pode ser relevante medir a diferença temporal entre transações consecutivas para o mesmo cartão de crédito, na expectativa de que esta informação revele algum padrão sobre a legitimidade (ou não) de uma transação. Assim, pode ser derivado um atributo *days_since_last_transaction* com a distância temporal (em dias) de cada transação em relação à última transação com o mesmo cartão de crédito.

In [ ]:
new_transactions = new_transactions.sort_values(by = ['cc_num', 'unix_time'])

new_transactions['days_since_last_transaction'] = new_transactions.groupby('cc_num')['unix_time'].diff()
new_transactions['days_since_last_transaction'] = round(new_transactions['days_since_last_transaction'] / (60 * 60 * 24), 3)
new_transactions['days_since_last_transaction'] = new_transactions['days_since_last_transaction'].fillna(0)

new_transactions.head()

Em quarto e último lugar, pode haver interesse em contabilizar o número de transações anteriormente efetuadas por cada cliente num dado comerciante, de modo a auxiliar na classificação de uma transação como fraudulenta ou legítima. Nesse sentido, cria-se o atributo *merchant_transactions_count*.

In [ ]:
new_transactions = new_transactions.sort_values(by = ['cc_num', 'merchant', 'unix_time'])

new_transactions['merchant_transactions_count'] = new_transactions.groupby(['cc_num', 'merchant']).cumcount()

new_transactions.head()

Assim, espera-se que os dados traduzam informação mais significativa para o modelo, tendo em vista um melhor desempenho.

### Integração dos Dados

De forma a integrar todos os quatro conjuntos de dados num único conjunto de dados com toda a informação, deve ser feita a junção dos dados pelas chaves primárias de cada tabela.

In [ ]:
data = new_transactions.merge(new_merchants, on = 'merchant', how = 'outer').merge(new_customers.merge(new_cities, on = 'city', how = 'outer'), on = 'cc_num', how = 'outer')

data.head()

Antes de mais, deve ser criado o atributo *age* com a idade de cada cliente no momento de cada transação por ele efetuada, tal como determinado anteriormente.

In [ ]:
data['age'] = pd.to_datetime(data['unix_time'], unit = 's').dt.year - pd.to_datetime(data['dob']).dt.year

data = data.drop('dob', axis = 1)

data.head()

Posto isto, o atributo *unix_time* deixa de ser necessário, devendo, portanto, ser removido.

In [ ]:
data = data.drop('unix_time', axis = 1)

data.head()

Após a junção, o atributo *cc_num* também já pode ser eliminado do conjunto de dados, porque essa coluna tinha como único propósito permitir a junção entre clientes e transações, sendo irrelevante para a determinação do valor da variável-alvo.

In [ ]:
data = data.drop('cc_num', axis = 1)

data.head()

Ao fazer a junção dos dados, podem ter surgido valores em falta, resultantes da operação de junção externa (*outer join*). Foi feita uma operação de junção externa em vez de junção interna (*inner join*) porque, caso contrário, seriam perdidos dados potencialmente relevantes: tantos quantas as linhas com pelo menos um valor em falta.

Assim, agora devem ser averiguados os valores em falta e tomadas as decisões apropriadas.

In [ ]:
data.isna().sum()

Tal como é possível visualizar, as únicas colunas com valores em falta são *city_pop* e *state*. Em teoria, todos estes valores em falta dever-se-ão à cidade *Test City* já mencionada, que não tem informação no conjunto de cidades.

In [ ]:
data[data['city'] == 'Test City'].shape[0]

Efetivamente, confirma-se que todos os valores em falta para os atributos *city_pop* e *state* se devem à cidade *Test City*. Tendo em conta que a relevância destes atributos para a determinação do valor da variável-alvo é extremamente duvidosa, pode procurar confirmar-se esse facto para decidir a ação a tomar.

In [ ]:
data['city_pop'].corr(data['is_fraud'])

Ora, a correlação entre a população da cidade à qual pertence um cliente e a legitimidade de uma transação que esse cliente efetua é praticamente nula, pelo que a coluna *city_pop* pode ser removida, sem considerável perda de informação.

In [ ]:
data = data.drop('city_pop', axis = 1)

In [ ]:
data.groupby('state')['is_fraud'].mean()

Similarmente, também não se verifica qualquer relação relevante entre a coluna *state* e a variável-alvo, pelo que este atributo também deverá ser removido.

In [ ]:
data = data.drop('state', axis = 1)

data.isna().sum()

Assim, todos os dados estão integrados num único conjunto, que, após a formatação, será fornecido ao modelo.

### Análise Descritiva

Tendo o conjunto de dados completamente limpo e integrado, pode ainda realizar-se mais alguma análise descritiva, tendo em vista a obtenção de eventuais informações relevantes, principalmente resultantes da integração dos conjuntos de dados e do processo de engenharia de funcionalidades.

Em primeiro lugar, pode averiguar-se a existência de alguma correlação significativa entre as novas variáveis e a variável-alvo, isto é, procurar responder à questão **"Existe algum atributo determinante para a classificação de uma transação como fraudulenta?"**.

In [ ]:
plt.figure(figsize = (10, 10))

correlation = data.select_dtypes(['float', 'int']).corr()
sb.heatmap(correlation, annot = True, cmap = 'coolwarm', mask = np.triu(np.ones_like(correlation, dtype = bool)), vmin = -1, vmax = 1)
contextualize('Correlograma de Transações', '', '')

Efetivamente, não se identifica nenhuma correlação significativa entre as variáveis numéricas e a variável-alvo.

A par desta análise, pode verificar-se também se existe algum padrão relevante na proporção de transações fraudulentas, relativamente aos diferentes valores que as variáveis categóricas podem assumir. Deste modo, elencam-se de seguida as questões que se procuram responder, através da visualização de gráficos/tabelas.

In [ ]:
def plot_table(category, title, xlabel, ylabel):
    plt.figure(figsize = (5, 5))
    table = pd.crosstab(data[category], data['is_fraud'])
    row_sum = table.sum(axis = 1)
    percentages = (table.T / row_sum * 100).T
    annotations = [['{} ({}%)'.format(count, round(percentage)) for percentage, count in zip(row, table_row)] for table_row, row in zip(table.values, percentages.values)]
    sb.heatmap(percentages, annot = annotations, fmt = '', cmap = 'coolwarm')
    contextualize(title, xlabel, ylabel)

**"Há algum padrão na distribuição de transações fraudulentas por emprego?"**

In [ ]:
plot_table('job', 'Ilegitimidade das Transações por Emprego', 'Ilegitimidade das Transações', 'Emprego')

Ao observar a tabela, conclui-se que a prevalência de transações fraudulentas é aproximadamente igual para todos os empregos dos clientes. Como tal, o atributo *job* não parece ser relevante para o modelo, pelo que não se justifica a sua manutenção nos dados a serem ingeridos pelo modelo.

In [ ]:
data = data.drop('job', axis = 1)

**"Há alguma relevância na categoria dos comerciantes para a determinação da legitimidade de uma transação?"**

In [ ]:
plot_table('category', 'Ilegitimidade das Transações por Categoria', 'Ilegitimidade das Transações', 'Categoria')

Nesta tabela, verifica-se que a proporção de transações fraudulentas é praticamente igual em cada categoria possível dos comerciantes, pelo que este atributo não será decisivo para a classificação de uma transação. Como tal, decide-se eliminar esta coluna dos dados a ingerir pelo modelo.

In [ ]:
data = data.drop('category', axis = 1)

**"Algum género tem mais tendência para efetuar transações fraudulentas?"**

In [ ]:
plot_table('gender', 'Ilegitimidade das Transações por Género', 'Ilegitimidade das Transações', 'Género')

Da visualização da tabela, evidencia-se que existe uma prevalência exatamente igual de transações fraudulentas nos clientes de géneros diferentes. Assim sendo, a informação do género será removida do modelo.

In [ ]:
data = data.drop('gender', axis = 1)

**"Há alguma cidade com maior proporção de transações fraudulentas?"**

In [ ]:
plot_table('city', 'Ilegitimidade das Transações por Cidade', 'Ilegitimidade das Transações', 'Cidade')

A tabela exposta demonstra que existe algum padrão na distribuição de transações fraudulentas/legítimas por cidade, particularmente na cidade *New York*, que tem uma proporção de transações fraudulentas consideravelmente inferior às restantes cidades.

Este facto poderá ser justificado devido à estrutura/organização dos Estados Unidos da América, em que diferentes cidades são regidas por diferentes leis (de acordo com o estado em que se inserem), pelo que pode ser uma informação relevante para o modelo.

Deste modo, em vez de manter o atributo *city*, opta-se pela transformação desta coluna numa nova coluna *city_NewYork* que indica se uma dada transação foi efetuada por um cliente da cidade *New York* (1) ou de outra cidade (0).

In [ ]:
data['city_NewYork'] = (data['city'] == 'New York').astype(int)

data = data.drop('city', axis = 1)

data.head()

Finalmente, a análise descritiva deve ser complementada por modelação descritiva, ou seja, pela realização de agrupamento (*clustering*) dos dados.

Nesse sentido, podem ser explorados dois métodos de agrupamento particional: os algoritmos *k-means* e *Density-Based Clustering of Applications with Noise* (DBSCAN). Não se considera relevante/pertinente realizar agrupamento hierárquico, visto que não é expectável que exista alguma hierarquia relevante entre eventuais grupos de valores, dado que o problema a tratar é de classificação binária.

Previamente ao agrupamento dos dados, estes devem ser normalizados através de *Min-Max Scaling*, de maneira a uniformizar as amplitudes das escalas dos diferentes atributos numéricos e evitar que os atributos de maior magnitude absorvam/dominem os atributos de menor magnitude.

Para além disso, antes de correr o algoritmo *k-means*, deve ainda ser determinado o número ideal de grupos, nomeadamente através do coeficiente de silhueta (*silhouette-based method*) e do método do cotovelo (*elbow method*). Assim, pode testar-se um número de grupos entre 2 (mínimo) e 10 (máximo).

A tentativa de agrupamento pretenderá responder à questão **"Existem grupos de transações por idade e quantia, isto é, há alguma tendência para uma determinada faixa etária fazer transações de uma dada quantia?"**, pelo que apenas se utilizarão as variáveis *age* e *amt*, devidamente normalizadas.

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['age', 'amt']])

k_range = range(2, 10)

silhouette_coefficients = []
inertias = []
for k in k_range:
    kmeans = KMeans(k, random_state = 10)
    labels = kmeans.fit_predict(scaled_data)
    silhouette_coefficient = silhouette_score(scaled_data, labels)
    silhouette_coefficients.append(silhouette_coefficient)
    inertias.append(kmeans.inertia_)

plt.figure(figsize = (20, 5))

plt.subplot(1, 2, 1)
plt.plot(k_range, silhouette_coefficients, marker = 'o')
contextualize('Número Ótimo de Grupos pelo Método do Coeficiente de Silhueta', 'Número de Grupos', 'Média do Coeficiente de Silhueta')

plt.subplot(1, 2, 2)
plt.plot(k_range, inertias, marker = 'o')
contextualize('Número Ótimo de Grupos pelo Método do Cotovelo', 'Número de Grupos', 'Inércia (Soma dos Erros Quadrados)')

plt.show()

Assim, conclui-se que o número ideal de grupos é 4, por ser o valor de *k* que retorna o coeficiente de silhueta mais elevado e por também ser o valor de *k* a partir do qual adicionar outro grupo não retorna uma inércia (soma dos erros quadrados) muito menor.

In [ ]:
kmeans = KMeans(4, random_state = 10)
kmeans_labels = kmeans.fit_predict(scaled_data)

dbscan = DBSCAN(eps = 0.05)
dbscan_labels = dbscan.fit_predict(scaled_data)

plt.figure(figsize = (20, 5))

plt.subplot(1, 3, 1)
plt.scatter(scaled_data[:, 0], scaled_data[:, 1], s = 10, c = kmeans_labels)
contextualize('Agrupamento K-Means', 'Idade Normalizada', 'Quantia Normalizada')

plt.subplot(1, 3, 2)
plt.scatter(scaled_data[:, 0], scaled_data[:, 1], s = 10, c = dbscan_labels)
contextualize('Agrupamento DBSCAN', 'Idade Normalizada', 'Quantia Normalizada')

plt.subplot(1, 3, 3)
scatterplot = plt.scatter(scaled_data[:, 0], scaled_data[:, 1], s = 10, c = data['is_fraud'])
contextualize('Agrupamento Real', 'Idade Normalizada', 'Quantia Normalizada')
plt.legend(*scatterplot.legend_elements(), title = 'is_fraud')

plt.show()

Efetivamente, através da análise dos grupos formados, quer pelo algoritmo *k-means*, quer pelo método DBSCAN, não se consegue extrair nenhuma informação relevante que responda de forma satisfatória à questão colocada anteriormente. A par disso, também não se encontra nenhuma ligação entre os grupos determinados através dos métodos de agrupamento e a distribuição real de transações fraudulentas e legítimas.

Deste modo, dá-se por concluída a análise descritiva dos dados.

### Formatação dos Dados

Como última etapa da fase de preparação de dados, os dados devem ser formatados para serem adequadamente ingeridos pelo modelo e para traduzirem adequadamente as grandezas que representam.

In [ ]:
data.dtypes

Dado que alguns modelos a testar necessitam que todos os atributos sejam numéricos, torna-se necessário codificar os atributos categóricos não ordinais, convertendo-os em atributos numéricos, nomeadamente através de *one-hot encoding*.

Note-se que a coluna *merchant* contém valores categóricos nominais e cerca 100 valores únicos/distintos, pelo que realizar *one-hot encoding* iria criar cerca de 100 novos atributos para o modelo tratar. Como tal, tendo em conta que o comerciante não foi considerado claramente relevante para a classificação de uma transação como fraudulenta ou legítima, opta-se por remover o atributo *merchant*, de modo a não aumentar drasticamente o número de colunas do modelo.

In [ ]:
formatted_data = data.drop('merchant', axis = 1)

formatted_data.dtypes

Assim, não restam colunas em que é necessário realizar *one-hot encoding*, dado que a coluna *city* já foi adequadamente tratada e transformada em *city_NewYork*.

In [ ]:
formatted_data = pd.get_dummies(formatted_data, drop_first = True)

formatted_data.dtypes

Antes de considerar os dados como prontos, podem ainda ser ordenadas as colunas alfabeticamente, de maneira a garantir que não existem inconsistências na ordem dos atributos.

In [ ]:
formatted_data = formatted_data.sort_index(axis = 1)

formatted_data.head()

Assim sendo, os dados estão, finalmente, prontos para serem ingeridos pelo modelo.

In [ ]:
formatted_data.to_csv('data/data.csv', index = False)

new_customers.to_csv('kaggle/customers.csv', index = False)
new_merchants.to_csv('kaggle/merchants.csv', index = False)